# Descarga y preparación del corpus de datos

METER TEXTO EXPLICATIVO AQUÍ !!!!!!!!!!!!!!!!!!!!!

## Importaciones

In [2]:
import librosa

import numpy as np
import datasets

from tqdm.auto import tqdm
import os

## Acceso al conjunto de datos de HuggingFace

METER TEXTO EXPLICATIVO AQUÍ !!!!!!!!!!!!!!!!!!!!!!!!!! (Y EN TODOS LADOS)

In [3]:
ccmusic_corpus = datasets.load_dataset("ccmusic-database/music_genre", name="default")

c:\Users\Usuario\anaconda3\envs\aine\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for ccmusic-database/music_genre contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccmusic-database/music_genre
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 1370 examples [00:00, 3124.94 examples/s]
Generating validation split: 171 examples [00:00, 4282.10 examples/s]
Generating test split: 172 examples [00:00, 7334.86 examples/s]


In [4]:
print(ccmusic_corpus)

DatasetDict({
    train: Dataset({
        features: ['audio', 'mel', 'fst_level_label', 'sec_level_label', 'thr_level_label'],
        num_rows: 1370
    })
    validation: Dataset({
        features: ['audio', 'mel', 'fst_level_label', 'sec_level_label', 'thr_level_label'],
        num_rows: 171
    })
    test: Dataset({
        features: ['audio', 'mel', 'fst_level_label', 'sec_level_label', 'thr_level_label'],
        num_rows: 172
    })
})


In [6]:
print(ccmusic_corpus['train'][0])

{'audio': {'path': 'C:\\Users\\Usuario\\.cache\\huggingface\\datasets\\downloads\\extracted\\9978c1aa27e41c465d1a0a120f523eae02b367b680fff33f401cee81e10d28c4\\audio\\2_non-classic\\11_rock\\21_Soft Rock\\6573efb8158239c2015abeaea6bb8f65.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 22050}, 'mel': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=496x369 at 0x19C52808110>, 'fst_level_label': 1, 'sec_level_label': 8, 'thr_level_label': 14}


## Generación de la estructura de directorios del corpus

* Función para la generación de las características:

In [ ]:
def store_features(audio_data, label_column, csv_file):

    # Extraer la información del audio (array numpy, sample rate, label)
    audio_array = audio_data["audio"]["array"]
    audio_sr = audio_data["audio"]["sr"]
    audio_label = audio_data[label_column]

    # Extraer las características del audio 

    # Dominio temporal
    # envelope = 
    rms = librosa.feature.rms(y=audio_array)
    zcr = librosa.feature.zero_crossing_rate(audio_array)

    # Dominio frecuencial
    # ber =
    spec_cent = librosa.feature.spectral_centroid(y=audio_array, sr=audio_sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=audio_array, sr=audio_sr)

    chroma_stft = librosa.feature.chroma_stft(y=audio_array, sr=audio_sr)
    rolloff = librosa.feature.spectral_rolloff(y=audio_array, sr=audio_sr)
    mfcc = librosa.feature.mfcc(y=audio_array, sr=audio_sr)

    # Generar fila para introducir en el CSV
    row = f"{audio_label},{np.mean(envelope)},{np.mean(rms)},{np.mean(zcr)},{np.mean(ber)},{np.mean(spec_cent)},{np.mean(spec_bw)},{np.mean(chroma_stft)},{np.mean(rolloff)}"
    for e in mfcc:
        row += f",{np.mean(e)}"

    # Escribir en el CSV
    with open(csv_file, "a") as f:
        f.write(row + "\n")

* Función para almacenar las anotaciones:

In [ ]:
def store_annotation(file_name, label_id, label_name, csv_file):
    with open(csv_file, "a") as f:
        f.write(f"{file_name},{label_id},{label_name}\n")

* Función para la generación del fichero wav:

In [ ]:
def store_audio_file(audio_data, audio_folder, label_column):


* Función para la generación del directorio de ficheros del corpus:

In [ ]:
def generate_corpus_files(corpus_data, label_column, corpus_folder):

    # Crear directorio principal
    os.makedirs(corpus_folder, exist_ok=True)

    # Almacenar ficheros de cada partición
    for partition in ["train", "validation", "test"]:

        # Crear subdirectorios para almacenar los audios
        os.makedirs(f"{corpus_folder}/{partition}/audios", exist_ok=True)

        # Crear fichero de anotaciones
        with open(f"{corpus_folder}/{partition}/annotations.csv", "w") as f:
            f.write("audio_file,label_id,label_name\n")

        # Crear fichero de características
        with open(f"{corpus_folder}/{partition}/features.csv", "w") as f:
            f.write("label,mean_envelope,mean_rms,mean_zcr,\
                    mean_ber,mean_spec_cent,mean_spec_bw,mean_chroma_stft,mean_rolloff,\
                    mean_mfcc1,mean_mfcc2,mean_mfcc3,mean_mfcc4,mean_mfcc5,mean_mfcc6,mean_mfcc7,mean_mfcc8,mean_mfcc9,mean_mfcc10,mean_mfcc11,mean_mfcc12,mean_mfcc13\n")
            

        for audio_data in corpus_data[partition]:

            # Guardar características en fichero CSV
            store_features(audio_data, label_column, f"{corpus_folder}/{partition}/features.csv")

            # Guardar anotación en fichero CSV
            store_annotation(audio_data, label_column, f"{corpus_folder}/{partition}/annotations.csv")
            
            # Guardar el audio en formato WAV
            store_audio_file(audio_data, f"{corpus_folder}/{partition}/audios")

* Carga de datos:

In [ ]:
generate_corpus_files(ccmusic_corpus, "fst_level_label", "ccmusic")

In [ ]:
generate_corpus_files(ccmusic_corpus, "sec_level_label", "ccmusic2")